# Würfelspiele (Python Übung)
Im Rahmen dieses Tutorials lernen wir den Umgang mit der Programmiersprache Python. Mit dieser Programmiersprache wird ein Großteil der KI-Modelle trainiert.

Hier lernen wir:
- das Arbeiten mit Zufall in Programmen
- das Verändern und Generieren von Noten mit Programmen
- das Laden von MIDI-Dateien in Python mit der music21 Library


## Importe
Um Midi-Dateien zu laden und für zufällige Zahlen des Würfelspiels benötigen wir einige Bibliotheken, die wir einfach importieren können. Klicke dazu auf das Play-Symbol in der linken oberen Ecke des Code-Blocks.

In [75]:
import music21
import random
import copy
from google.colab import files
import pandas as pd
!wget "https://github.com/langMatthias/ai-intro/raw/main/exercises/medium/01%20-%20Einf%C3%BChrung%20in%20Python/MOZART-W%C3%BCrfelspiel.mid" --quiet
!wget "https://github.com/langMatthias/ai-intro/raw/main/exercises/medium/01%20-%20Einf%C3%BChrung%20in%20Python/mozart_table.csv" --quiet

## Funktionen
In Python (wie auch in vielen anderen Programmiersprachen) kannst du sogenannte Funktionen definieren. Diese können dann mit unterschiedlichen Parametern aufgerufen werden.
Um eine MIDI-Datei zu laden, haben wir hier schon einmal eine Funktion vorbereitet. Sehe dir genau an, wie sie funktioniert. Wir nutzen die music21 Bibliothek um die MIDI-Datei zu laden. Als Parameter übergeben wir der Funktion den Pfad zur MIDI-Datei. Das Ergebnis ist eine music21 Score.

In [76]:
def load_midi(filepath):
    """
    Eine Funktion, die eine MIDI-Datei von einem Dateipfad lädt und als music21 Score zurück gibt.
    """
    score = music21.converter.parse(filepath)
    return score

Als nächstes nutzen wir die Funktion um die MIDI-Datei des Würfelspiels von Mozart zu laden.

In [77]:
mozart_score = load_midi("MOZART-Würfelspiel.mid")

## Anzahl der enthaltenen Takte
Um aus den in der MIDI-Datei enthaltenen Takten auszuwählen, müssen wir wissen wie viele Takte darin enthalten sind. Dazu implementieren wir die Funktion `get_number_of_measures` welche eine music 21 Score als Parameter übergeben bekommt und die Anzahl der in der Score enthaltenen Takte zurück gibt.

Die Takte erhalten wir als eine Liste die wir `measures` nennen. Dafür nutzen wir einen komplizierteren Aufruf, den du nicht im Detail verstehen musst. Um nun die Anzahl der Takte zu erhalten können wir den Python-Befehl `len()` nutzen. Versuche also mit Hilfe dieser Funktion die Länge der Takt-Liste zu erhalten. Damit unsere Funktion `get_number_of_measures` nun auch diesen Wert wieder zurück gibt müssen wir noch ein return statement einfügen. Schaffst du es, dass der folgende Code-Block folgenden Text ausgibt?

> In dieser MIDI-Datei wurden 176 Takte gefunden.

Für die Ausgabe dieses Textes nutzen wir übrigens die print() Funktion mit der wir in Python alles als Text ausgeben können.

In [78]:
def get_number_of_measures(score):
    """
    Diese Funktion erhält eine music21 Score und gibt die Anzahl der darin enthaltenen Takte zurück.
    """
    measures = score.parts[0].getElementsByClass('Measure')
    return len(measures)

num_measures = get_number_of_measures(mozart_score)
print(f"In dieser MIDI-Datei wurden {num_measures} Takte gefunden.")

In dieser MIDI-Datei wurden 176 Takte gefunden.


## Den richtigen Takt auswählen
Für das Würfelspiel benötigen wir jetzt eine Möglichkeit den richtigen Takt aus einer Score auszuwählen. Dafür haben wir schon einmal die Funktion `get_measure` vorbereitet.

Sehe dir diese Funktion genau an. Es fehlt genau der Teil, der den richtigen Takt auswählt und als `selected_measure` abspeichert. Wie könnten wir das lösen?

Tipp: Erinnere ich an die Liste an Takten, die wir für `get_number_of_measures` genutzt haben. Vielleicht können wir diese nutzen um daraus die Takte auszuwählen. Recherchiere, wie man in Python ein Element aus einer Liste auswählt.
Beachte hierbei noch, dass diese Funktion den ersten Takt mit dem Aufruf `get_measure(mozart_score, 1)` auswählen soll. Bei Python fängt die Zählung der Elemente in einer Liste aber bei 0 an. Wie können wir das lösen?

Für das `return` Statement nutzen wir dann noch die Python Bibliothek `copy` um nicht den Original-Takt sondern eine Kopie dessen zurück zu geben. Das machen wir um sicher zu gehen dass wir die Original-Takte nicht verändern.

In [79]:
def get_measure(score, measure_number):
    """
    Diese Funktion erhält eine music21 Score und eine Taktzahl und gibt eine Kopie des angegebenen Taktes zurück.
    Achtung: Wir müssen 1 von der angegeben Taktzahl subtrahieren, da die Zählung der Takte bei music21 mit 0 beginnt.
    """
    if measure_number <= 0:
      print("Die Zählung der Taktzahlen beginnt mit 1.")
      return None
    try:
      measures = score.parts[0].getElementsByClass('Measure')
      selected_measure = measures[measure_number-1]
      return copy.deepcopy(selected_measure)
    except:
      print(f"Takt {measure_number} konnte leider nicht ausgewählt werden, da die gegebene Score nur {get_number_of_measures(score)} Takte enthält.")
      return None


Nun können wir unsere Funktion aufrufen und testen, ob sie korrekt funktioniert. Dafür haben wir hier einen Test vorbereitet. Führe den folgenden Codeblock aus um zu testen, ob deine Implementierung korrekt funktioniert.

In [80]:
# Test
first_measure = get_measure(mozart_score, 1)
try:
  first_chord = first_measure.notes[0]
  assert first_chord.notes[0].pitch == music21.pitch.Pitch('F5'), "Es scheint, als wäre der falsche Takt geladen worden. Achte darauf, dass Takt 1 in der Liste der Takte als Element 0 gespeichert ist."
  print("Der Test war erfolgreich.")
except Exception as e:
  print(f"Der Test ist leider fehlgeschlagen. Verifiziere, dass die Funtion get_measure() einen music21 Measure zurückgibt.")
  print(e)

Der Test war erfolgreich.


Damit wir nun mehrere Takte mit ihren Taktzahlen auswählen können benötigen wir eine Funktion, die wir mit einer Score und einer Liste an Taktzahlen aufrufen können. Diese soll dann die score 21 Measures als Liste zurückgeben. Da wir hier mehrfach die gleiche Aufgabe ausführen wollen, eignet sich ein sogenannter for-Loop. Mache dich mit dem Konzept des for-loops vertraut (https://wiki.python.org/moin/ForLoop) und versuche dann die Funktion zu implementieren.

In [81]:
def select_measures(score, measure_numbers):
    """
    Eine Funktion, die mehrere Takte einer Score auswählt und als Liste zurück gibt.
    Die Taktzahlen der einzelnen Takte werden auch als Liste übergeben.
    """
    selected_measures = []
    for measure_number in measure_numbers:
      selected_measures.append(get_measure(score, measure_number))
    return selected_measures

Auch für diese Funktion haben wir einen kleinen Test vorbereitet.

In [82]:
try:
  test_measures = [1,42,123, 176]
  test_selection = select_measures(mozart_score, test_measures)
  for test_measure, selected_measure in zip(test_measures, test_selection):
    assert test_measure == selected_measure.number, f"Es sieht aus, als wäre ein falscher Takt ausgewählt worden. Erwartet wurde Takt {test_measure}, ausgewählt wurde allerdings Takt {selected_measure.number}"
  print("Der Test war erfolgreich.")
except Exception as e:
    print(f"Der Test ist leider fehlgeschlagen. Verifiziere, dass die Funtion get_measure() einen music21 Measure zurückgibt.")
    print(e)

Der Test war erfolgreich.


Nun können wir aus der Liste der ausgewählten Takte eine music21 Score generieren um diese dann als MIDI-Datei zu speichern. Dafür erstellen wir eine neue, leere music21 Score und einen music21 Part. Nutze nun die Funktion `part.append()` um alle Takte dem Part zuzuweisen.

In [83]:
def create_score(measures, time_signature=None):
    """
    Erstellt eine music21 Score aus einer Liste an music21 Takten.
    Damit die Noten korrekt in Notations-Software angezeigt werden, kann hier auch die Taktart angegeben werden (z.B. '3/8' für Mozarts Würfelspiel)
    """
    score = music21.stream.Score()
    part = music21.stream.Part()
    for measure in measures:
        part.append(measure)
    score.append(part)
    if time_signature is not None:
      score.parts[0].getElementsByClass('Measure')[0].timeSignature = music21.meter.TimeSignature(time_signature)
    return score

Nun können wir unsere ersten Test-Takte zu einer Score kombinieren. Ergänze den folgenden Codeblock und finden einen Weg, dir die generierte Score anzuhören.

In [84]:
# creating a music21 Score from our test_selection
test_score = create_score(test_selection, time_signature='3/8')
# Listening to the test_score
test_score.show('midi')

Um die generierte music21 Score nun auch als MIDI zu speichern fehlt uns noch eine Funktion `save_midi`. Finde heraus, wie man MIDI Files mit music21 speichert und speichere die MIDI-Datei mit dem Namen `test.mid`. Du findest die gespeicherte Datei dann, indem du in der linken Spalte auf das Ordner-Symbol klickst. Dort kannst du dann auch das File herunterladen und auf deinem Computer weiter bearbeiten.

In [85]:
def save_midi(score, filepath):
    """
    Speichert eine music21 Score am angegebenen Dateipfad
    """
    score.write('midi', fp=filepath)

In [86]:
save_midi(create_score(test_selection), 'test.mid')

## Zufall
Für das Würfelspiel fehlt uns jetzt noch der Würfel. Hierbei lassen wir einfach den Computer für uns würfeln. Nutze die Python-Bibliothek random um eine zufällige Zahl mit der Funktion `roll_dice()` auszugeben.

Bei dieser Funktion gibt es die Besonderheit, dass der Parameter `number_of_faces` mit einem Standard-Wert (default) von 6 übergeben wird. Diesen Parameter können wir also explizit setzen und zum Beispiel einen 12-seitigen Würfel simulieren. Wir müssen den Parameter aber nicht setzen. Wird er nicht mit übergeben, so wird der Default-Wert von 6 genutzt. Versuche nun die Funktion `roll_dice()` zu ergänzen.

In [87]:
def roll_dice(number_of_faces=6):
    """
    Simuliert das Werfen eines Würfels. Dabei kann die Anzahl der Seiten angegeben werden.
    """
    return random.randint(1, number_of_faces)

Auch dafür haben wir wieder einen kleinen Test geschrieben.

In [88]:
faces = set()
for i in range(1000):
  roll = roll_dice()
  assert roll > 0 and roll < 7, f"Dein Würfel hat eine unerwartete Zahl gewürfelt: {roll}."
  faces.add(roll)
assert faces == {1,2,3,4,5,6}, f"Dein Würfel konnte folgende Zahlen nicht würfeln: {set([1,2,3,4,5,6]).difference(faces)}."
print("Der Test war erfolgreich.")

Der Test war erfolgreich.


Um schließlich das Würfelspiel komplett zu machen, müssen wir noch eine Funktion schreiben, die den Würfel mehrfach hintereinander wirft. Implementiere die Funktion `wuerfelspiel()`, die das mehrmalige Werfen eines Würfels simuliert und das Ergebnis als Liste zurück gibt.

In [89]:
def wuerfelspiel(number_of_repetitions, number_of_faces=6):
    """
    Simuliert das wiederholte Werfen eines Würfels und gibt die geworfenen Würfe als Liste zurück.
    Lösung mit List Comprehension.
    """
    return [roll_dice(number_of_faces=number_of_faces) for _ in range(number_of_repetitions)]

def wuerfelspiel(number_of_repetitions, number_of_faces=6):
    """
    Simuliert das wiederholte Werfen eines Würfels und gibt die geworfenen Würfe als Liste zurück.
    Lösung mit For-Loop.
    """
    rolls = []
    for i in range(number_of_repetitions):
      rolls.append(roll_dice(number_of_faces=number_of_faces))
    return rolls

Auch hierfür gibt es wieder einen Test.

In [90]:
rolls = wuerfelspiel(1000)
assert type(rolls) is list
possibilities = set(rolls)
possible_throws = {1,2,3,4,5,6}
assert len(possibilities - possible_throws) == 0, f"Dein Würfel konnte mehr Zahlen als erwartet würfeln {possibilities.difference(possible_throws)}"
assert possibilities == possible_throws, f"Dein Würfel konnte folgende Zahlen nicht würfeln: {possible_throws.difference(possibilities)}."
print("Der Test war erfolgreich.")

Der Test war erfolgreich.


## Jetzt wird gewürfelt!
Um das Würfelspiel komplett durchzuspielen, müssen wir nun einige Schritte nacheinander ausführen.

1. Wir generieren eine Liste mit einer beliebigen Anzahl an Würfel-Würfen (Entscheide selbst, wie lange dein Stück werden soll)
2. Wir wählen die jeweiligen Takte aus der Score aus
3. Wir generieren eine neue Score aus den ausgewählten Takten
4. Wir speichern diese Score als MIDI-Datei

Führe nun diese Schritte nacheinander aus.

In [91]:
# Generate list of dice rolls
wuerfelspiel_selection = wuerfelspiel(16)
print(f"Es wurde {wuerfelspiel_selection} gewürfelt.")
# Get the music21 Measures according to the dice rolls
selected_measures = select_measures(mozart_score, wuerfelspiel_selection)
# Create a new Score from the selected measures
new_score = create_score(selected_measures, time_signature='3/8')
# Listen to the MIDI file
new_score.show('midi')
# Save the MIDI file
save_midi(new_score, 'wuerfelspiel.mid')

Es wurde [5, 3, 5, 2, 4, 6, 4, 1, 1, 3, 4, 6, 6, 2, 2, 6] gewürfelt.


Diese Auswahl basiert nur auf den ersten 6 Takten der MIDI-Datei. Um alle zu nutzen, könnten wir aber einfach einen 176-seitigen Würfel simulieren. Generiere nun ein Stück basierend auf allen 176 Takten.

In [92]:
# Generate list of dice rolls
wuerfelspiel_selection = wuerfelspiel(16, number_of_faces=176)
print(f"Es wurde {wuerfelspiel_selection} gewürfelt.")
# Get the music21 Measures according to the dice rolls
selected_measures = select_measures(mozart_score, wuerfelspiel_selection)
# Create a new Score from the selected measures
new_score = create_score(selected_measures, time_signature='3/8')
# Listen to the MIDI file
new_score.show('midi')
# Save the MIDI file
save_midi(new_score, 'extended_wuerfelspiel.mid')

Es wurde [63, 131, 65, 40, 77, 6, 14, 126, 104, 8, 23, 153, 174, 46, 79, 33] gewürfelt.


## Mozarts Würfelspiel
Mozarts Würfelspiel ist allerdings noch ein bisschen komplexer. Zum Einen nutzt er zwei Würfel, zum anderen werden hier die Takte nicht direkt nach den Augenzahlen ausgewählt, sondern aus einer Tabelle. Somit ergibt ein Würfelwurf je nachdem wann er fällt, ein anderes Ergebnis. Die Tabelle von Mozart kannst du dir mit dem folgenden Codeblock anzeigen lassen.

In [93]:
mozart_table = pd.read_csv('mozart_table.csv', index_col=0)
mozart_table.columns = mozart_table.columns.astype(int)
display(mozart_table)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
2,96,22,141,41,105,122,11,90,70,121,26,9,112,49,109,14
3,32,6,128,69,146,46,134,81,117,39,126,56,174,18,116,83
4,69,95,158,19,159,55,110,24,66,139,15,132,73,58,145,79
5,40,17,113,85,161,2,159,100,90,176,7,34,67,160,52,170
6,148,74,163,45,80,97,36,107,25,143,64,125,76,136,1,93
7,104,157,27,167,154,68,118,91,138,71,150,29,101,162,23,151
8,152,60,171,53,99,133,21,127,16,155,57,175,43,168,89,172
9,119,84,114,50,140,86,169,94,120,88,48,166,51,115,72,111
10,98,142,42,156,75,129,62,123,65,77,19,82,137,38,149,8
11,3,87,165,61,135,47,147,33,102,4,31,164,144,59,173,78


Die Tabelle ist wie folgt zu verstehen:

Wenn im 1. Takt eine 2 gewürfelt wird, wird Takt 96 ausgewählt. Wird im 2. Takt dann wieder eine 2 gewürfelt, würde allerdings Takt 22 ausgewählt werden.

Nun benötigen wir also zusätzlich zu den oben erstellten Funktionen, eine weitere Funktion, die die Auswahl aus der Tabelle übernimmt. Dafür solltest du du dich mit pandas Dataframes vertraut machen und herausfinden, wie du den richtigen Wert aus der Tabelle auswählst: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

In [94]:
def get_measure_number_with_table(roll, roll_number, table):
    """
    Diese Funktion erhält eine Würfelzahl, die Angabe der wievielte Wurf diese Zahl betrifft und eine Tablle aus der ausgewählt werden soll und gibt eine Kopie des angegebenen Taktes zurück.
    """
    measure_number = table.loc[roll, roll_number]
    return measure_number

Der Aufruf dieser Funktion mit `get_measure_number_with_table(2, 1, mozart_table)` sollte also 96 ergeben. Hier wieder ein Test.

In [95]:
first_test_value = get_measure_number_with_table(2, 1, mozart_table)
assert first_test_value == 96, f"get_measure_number_with_table() funktioniert scheinbar noch nicht richtig. Erwartet wurde 96, die Funktion gab aber {first_test_value}"
second_test_value = get_measure_number_with_table(10, 3, mozart_table)
assert second_test_value == 42, f"get_measure_number_with_table() funktioniert scheinbar noch nicht richtig. Erwartet wurde 42, die Funktion gab aber {second_test_value}"
print("Der Test war erfolgreich.")

Der Test war erfolgreich.


Da wir die Würfel mehrmals werfen werden, benötigen wir noch eine Funktion, die uns eine ganze Liste an Würfen in Taktzahlen übersetzt.
Hierbei können wir unsere Funktion `get_measure_number_with_table()` wieder verwenden. Allerdings benötigen wir für den Aufruf dieser Funktion auch die Angabe, um den wievielten Wurf es sich dabei handelt. Mache dich dafür mit der `enumerate()`-Funktion vertraut (https://docs.python.org/3/library/functions.html#enumerate)

Implementiere nun die Funktion `translate_rolls()`:

In [96]:
def translate_rolls(rolls, table):
    """
    Diese Funktion übersetzt eine Liste an Würfel-Ergebnissen in Taktzahlen einer Tabelle. Sie erhält die Würfe als Liste und die Tabelle als pandas DataFrame und gibt die übersetzten Taktzahlen wieder als Liste zurück.
    """
    assert len(rolls) <= len(table.columns), f"Die Tabelle ist zu kurz für die Anzahl der Würfel-Würfe."
    measures = []
    for i, roll in enumerate(rolls):
      measures.append(get_measure_number_with_table(roll, i+1, mozart_table))
    return measures

Auch hierfür ein Test:

In [97]:
test_rolls = [11, 3, 10, 7]
test_measures = [3, 6, 42, 167]
translated_measures = translate_rolls(test_rolls, mozart_table)
assert test_measures == translated_measures, f"Die Funktion hat die Takte leider falsch übersetzt. Erwartet waren {test_measures}, erhalten wurde aber {translated_measures}"
print("Der Test war erfolgreich.")

Der Test war erfolgreich.


Mit `get_measure()` können wir dann wie oben auch, die jeweiligen Takte auswählen und eine neue MIDI-Datei erstellen. Letztlich fehlt nur noch eine Funktion, die das Würfeln mit zwei Würfeln simuliert. Ergänze die folgende Funktion um jeweils mit mit mehreren Würfeln zu würfeln. Dabei schreiben wir die Funktion so, dass sie zusätzlich zu den Paramtern von `wuerfelspiel()`, auch eine beliebige Anzahl an Würfeln unterstützt.

Tipp: Hier können wir die Funktion `roll_dice()` wieder verwenden.

In [98]:
def roll_multiple_dice(number_of_dice, number_of_rolls, number_of_faces=6):
    """
    Mit dieser Funktion kann das Werfen mehrerer Würfel simuliert werden. Dafür erhält die Funktion als Input die Anzahl der Würfel, die Anzahl der Würfe und optional die Anzahl der Würfelseiten.
    Als Ergebnis liefert die Funktion eine Liste aller Würfe, wobei pro Wurf die Summe aller Würfel gespeichert wird.
    Lösung mit List Comprehension.
    """
    return [sum([roll_dice(number_of_faces=number_of_faces) for _ in range(number_of_dice)]) for _ in range(number_of_rolls)]


def roll_multiple_dice(number_of_dice, number_of_rolls, number_of_faces=6):
    """
    Mit dieser Funktion kann das Werfen mehrerer Würfel simuliert werden. Dafür erhält die Funktion als Input die Anzahl der Würfel, die Anzahl der Würfe und optional die Anzahl der Würfelseiten.
    Als Ergebnis liefert die Funktion eine Liste aller Würfe, wobei pro Wurf die Summe aller Würfel gespeichert wird.
    Lösung mit For-Loop.
    """
    rolls = []
    for i in range(number_of_rolls):
      sum = 0
      for j in range(number_of_dice):
        roll = roll_dice(number_of_faces=number_of_faces)
        sum += roll
      rolls.append(sum)
    return rolls

Und auch hier wieder ein Test:

In [99]:
rolls = roll_multiple_dice(2,1000)
assert type(rolls) is list
possibilities = set(rolls)
possible_throws = {2,3,4,5,6,7,8,9,10,11,12}
assert len(possibilities - possible_throws) == 0, f"Dein Würfel konnte mehr Zahlen als erwartet würfeln {possibilities.difference(possible_throws)}"
assert possibilities == possible_throws, f"Dein Würfel konnte folgende Zahlen nicht würfeln: {possible_throws.difference(possibilities)}."
print("Der Test war erfolgreich.")

Der Test war erfolgreich.


Nun kannst du mit dieser Funktion wieder
1. mehrere Würfel-Würfe simulieren,
2. diese dann mit Mozarts Tabelle übersetzen
3. und eine MIDI-Datei der ausgewürfelten Takte erstellen

In [100]:
# Simulate multiple dice rolls with two dice
mozarts_wuerfelspiel_selection = roll_multiple_dice(2, 16)
print(f"Es wurde {mozarts_wuerfelspiel_selection} gewürfelt.")
# Translate measure numbers with table
translated_selection = translate_rolls(mozarts_wuerfelspiel_selection, mozart_table)
print(f"Es wurden die folgenden Takte ausgewählt: {translated_selection}")
# Get the music21 Measures according to the dice rolls
selected_measures = select_measures(mozart_score, translated_selection)
# Create a new Score from the selected measures
new_score = create_score(selected_measures, time_signature='3/8')
# Listen to the MIDI file
new_score.show('midi')
# Save the MIDI file
save_midi(new_score, 'mozarts_wuerfelspiel.mid')

Es wurde [4, 7, 5, 7, 12, 11, 3, 10, 5, 3, 4, 5, 11, 9, 8, 5] gewürfelt.
Es wurden die folgenden Takte ausgewählt: [69, 157, 113, 167, 28, 47, 134, 123, 90, 39, 15, 34, 144, 115, 89, 170]


Damit haben wir das Ende dieses Tutorials erreicht.
Experimentiere mit eigenen Ideen oder MIDIs. Wie könnte man zum Beispiel die Logik der Taktauswahl verändern?

## Eigene MIDI-Dateien hochladen
Du kannst auch selbst MIDI-Dateien hochladen und dein eigenes Würfelspiel bauen.

Um eine MIDI-Datei zu verarbeiten, muss sie zunächst auf den Server hochgeladen werden. Dafür kannst du den folgenden Code-Block nutzen oder einfach links auf das Ordner-Symbol klicken und die Files auf den Server ziehen.

Wähle eine eigene MIDI-Datei aus, mit der du experimentieren möchtest. Dies könntest du zum Beispiel mit [MuseScore](https://musescore.org/de/download) erstellen.

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

print()
print(f"Die Datei {filename} wurde erfolgreich hochgeladen und der Name dieser Datei wurde in der Variable filename gespeichert.")